In [1]:
# Automatically install dependencies from requirements.txt
import sys
import subprocess

# Define a function to install packages
def install_requirements():
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
        print("All packages installed successfully.")
    except subprocess.CalledProcessError as e:
        print("An error occurred while installing packages:", e)

# Run the function
install_requirements()


Defaulting to user installation because normal site-packages is not writeable
All packages installed successfully.


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Lists to hold quote information
quotes_list = []
authors_list = []
tags_list = []
born_list = []
desc_list = []

# Base URL for quotes and author pages
base_url = "https://quotes.toscrape.com/"
page_url = "https://quotes.toscrape.com/page/{}/"

# Loop over all pages
page_number = 1
while True:
    response = requests.get(page_url.format(page_number))
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all quotes on the current page
    quotes = soup.find_all("div", class_="quote")
    if not quotes:
        print("No more quotes found, stopping.")
        break  # Exit the loop if no quotes found (end of pagination)

    for quote in quotes:
        # Extract quote text
        text = quote.find("span", class_="text").get_text(strip=True)
        # Extract author
        author = quote.find("small", class_="author").get_text(strip=True)
        # Extract tags
        tags = [tag.get_text(strip=True) for tag in quote.find_all("a", class_="tag")]

        # Access the author's detail page for additional info
        author_url = base_url + quote.find("a")["href"]
        author_response = requests.get(author_url)
        author_soup = BeautifulSoup(author_response.text, "html.parser")

        # Extract author's birth date and description
        born_date = author_soup.find("span", class_="author-born-date").get_text(strip=True)
        born_location = author_soup.find("span", class_="author-born-location").get_text(strip=True)
        born = f"{born_date}, {born_location}"
        description = author_soup.find("div", class_="author-description").get_text(strip=True)

        # Append data to lists
        quotes_list.append(text)
        authors_list.append(author)
        tags_list.append(", ".join(tags))
        born_list.append(born)
        desc_list.append(description)

        # Be respectful with delays for author page requests
        time.sleep(1)

    # Move to the next page
    page_number += 1

# Create a DataFrame from collected data
data = {
    "Quote": quotes_list,
    "Author": authors_list,
    "Tags": tags_list,
    "Born": born_list,
    "Description": desc_list
}
df = pd.DataFrame(data)
